In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("drug200_v2.csv")
df

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [3]:
df[df["BP"].isna()]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
14,50,F,NaN,HIGH,12.703,drugX
27,49,F,NaN,NORMAL,9.381,drugX
42,50,M,NaN,NORMAL,15.790,DrugY


In [4]:
#fill empty rows in BP column with the most frequently appearing value from records within the same age range(49-51)
female_mode = df[(df["Age"] >= 49) & (df["Age"] <=51)& (df["Sex"] =="F")]["BP"].mode().iat[0]
male_mode = df[(df["Age"] >= 49) & (df["Age"] <=51)& (df["Sex"] =="M")]["BP"].mode().iat[0]


In [5]:
df.loc[(df["Sex"]=="M") & (df["BP"].isna())] = df.loc[(df["Sex"]=="M") & (df["BP"].isna())].fillna(male_mode)
df.loc[(df["Sex"]=="F") & (df["BP"].isna())] = df.loc[(df["Sex"]=="F") & (df["BP"].isna())].fillna(female_mode)

In [6]:
df[df["BP"].isna()]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug


In [10]:
stdev = df["Na_to_K"].std()
mean = df["Na_to_K"].mean()
df["Na_to_K_z_normalised"] = df["Na_to_K"].apply(lambda x: (x-mean)/stdev)
print("Mean of normalised z-score of Na_to_K:", df["Na_to_K_z_normalised"].mean())
print("Variance of normalised z-score of Na_to_K:", df["Na_to_K_z_normalised"].var())

Mean of normalised z-score of Na_to_K: 1.397770788003072e-15
Variance of normalised z-score of Na_to_K: 1.0000000000000004


In [20]:
df=df.drop(columns=["bin"])

In [24]:
print(df["Na_to_K_z_normalised"].min())
print(df["Na_to_K_z_normalised"].max())

-1.3587410610891604
3.06791963387489


In [41]:
bins = [-1.5, -1, 0, 1, 2, 3.1]
labels = [1,2,3,4]
df["bin"] = pd.qcut(df["Na_to_K_z_normalised"], 4)
df

,Age,Sex,BP,Cholesterol,Na_to_K,Drug,Na_to_K_z_normalised,test,bin
0,23,F,HIGH,HIGH,25.355,DrugY,1.283302,1.283302,"(0.456, 3.068]"
1,47,M,LOW,HIGH,13.093,drugC,-0.414106,-0.414106,"(-0.781, -0.297]"
2,47,M,LOW,HIGH,10.114,drugC,-0.826484,-0.826484,"(-1.3599999999999999, -0.781]"
3,28,F,NORMAL,HIGH,7.798,drugX,-1.147084,-1.147084,"(-1.3599999999999999, -0.781]"
4,61,F,LOW,HIGH,18.043,DrugY,0.271114,0.271114,"(-0.297, 0.456]"
...,...,...,...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC,-0.625348,-0.625348,"(-0.781, -0.297]"
196,16,M,LOW,HIGH,12.006,drugC,-0.564578,-0.564578,"(-0.781, -0.297]"
197,52,M,NORMAL,HIGH,9.894,drugX,-0.856938,-0.856938,"(-1.3599999999999999, -0.781]"
198,23,M,NORMAL,NORMAL,14.020,drugX,-0.285783,-0.285783,"(-0.297, 0.456]"


In [40]:
df["bin"].value_counts()

1    50
2    50
3    50
4    50
Name: bin, dtype: int64